In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
data_preview = {name: df.head() for name, df in dataframes.items()}
data_preview

In [ ]:
# Preprocess each dataframe to handle non-numeric data and format inconsistencies
processed_dataframes = {}

for name, df in dataframes.items():
    # Remove commas and convert columns to numeric where applicable
    for col in df.columns[1:]:  # Skip the AREA column
        df[col] = df[col].replace(',', '', regex=True).apply(pd.to_numeric, errors='coerce')

    # Drop non-numeric columns
    numeric_df = df.select_dtypes(include=['number'])

    processed_dataframes[name] = numeric_df

# Apply scaling to the processed numeric data
scaled_dataframes = {
    name: pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    for name, df in processed_dataframes.items()
}

# Save and provide the scaled data for review
for name, scaled_df in scaled_dataframes.items():
    scaled_df.to_csv(f"{name}_scaled.csv", index=False)

scaled_dataframes


In [ ]:
from sklearn.impute import SimpleImputer

# Impute missing values with the mean for both features and target
imputer = SimpleImputer(strategy="mean")

X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)
y_imputed = pd.Series(imputer.fit_transform(y.values.reshape(-1, 1)).flatten(), index=y.index)

# Split the dataset into training and testing sets again
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed, test_size=0.2, random_state=42)

# Train the Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Predict on the test set
y_pred = ridge_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)

ridge_model, mse

In [ ]:
# Calculate the R^2 score (accuracy metric for regression models)
r2_score = ridge_model.score(X_test, y_test)
r2_score


In [ ]:
# 실제값
# Combine the predictions with the corresponding years and areas for better understanding
results = target_data.iloc[:, :].copy()  # Include the 'AREA' and year information

print(results)

In [ ]:
# 예측값
# Reshape the data for year-wise predictions
# Ensure the target data matches the structure for year-wise prediction
X_yearly = X_imputed.copy()
y_yearly = target_numeric.copy()

# Re-train the model to focus on year-wise relationships
ridge_model_yearly = Ridge(alpha=1.0)
ridge_model_yearly.fit(X_yearly, y_yearly)

# Predict birth population for each year (2007 to 2023)
predicted_births_yearly = ridge_model_yearly.predict(X_yearly)

# Combine predictions with the corresponding areas and years
results_yearly = target_data.iloc[:, :1].copy()  # Start with the 'AREA' column
results_yearly[y_yearly.columns] = predicted_births_yearly

print(results_yearly)